# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data

# Order of play

DAY 1: Data Curation  
DAY 2: Data Pre-processing  
DAY 3: Evaluation, Baselines, Traditional ML  
DAY 4: Deep Learning and LLMs  
DAY 5: Fine-tuning a Frontier Model  

## DAY 5: Fine-tuning a Frontier Model

Now we will use OpenAI's API to fine-tune our own private variant of GPT-4.1-nano

In [1]:
# imports

import os
import re
import json
from dotenv import load_dotenv
from huggingface_hub import login
from openai import OpenAI
from pricer.items  import Item
from pricer.evaluator import evaluate

In [2]:
# environment

LITE_MODE = True

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [4]:
openai = OpenAI()

# Data size

OpenAI recommends fine-tuning with a small population of 50-100 examples

I'm going to go with 20,000 points.

This cost me $3.42 - you should stick with 100 examples and the cost will be minimal!

In [5]:
# OpenAI recommends fine-tuning with populations of 50-100 examples
# But as our examples are very small, I'm suggesting we go with 100 examples (and 1 epoch)


fine_tune_train = train[:100]
fine_tune_validation = val[:50]

In [6]:
len(fine_tune_train)

100

# Step 1

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [7]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
        {"role": "assistant", "content": f"${item.price:.2f}"}
    ]

In [8]:
messages_for(fine_tune_train[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \nCategory: Home Hardware  \nBrand: Schlage  \nDescription: A single‑piece oil‑rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \nDetails: Designed for a 4" minimum center‑to‑center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation.'},
 {'role': 'assistant', 'content': '$64.30'}]

In [9]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [10]:
print(make_jsonl(train[:3]))

{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \nCategory: Home Hardware  \nBrand: Schlage  \nDescription: A single\u2011piece oil\u2011rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \nDetails: Designed for a 4\" minimum center\u2011to\u2011center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation."}, {"role": "assistant", "content": "$64.30"}]}
{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Mini Electric Air Duster Fan  \nCategory: Electronics  \nBrand: Kica  \nDescription: Ultra\u2011compact 86,000\u202fRPM electric air duster with 11\u202fm/s wind speed for precise cleaning and inflation.  \nDetails: Powered by a 9.99\u202fWh motor, adjustable in four speed levels, it uses three 

In [11]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [12]:
write_jsonl(fine_tune_train, "jsonl/fine_tune_train.jsonl")

In [13]:
write_jsonl(fine_tune_validation, "jsonl/fine_tune_validation.jsonl")

In [14]:
with open("jsonl/fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [15]:
train_file

FileObject(id='file-Js35KiuAFgARS2Es9o1DZG', bytes=55120, created_at=1769673666, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [16]:
with open("jsonl/fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [17]:
validation_file

FileObject(id='file-AfCR9zBTapjJ4vjssunxVM', bytes=27637, created_at=1769673671, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

https://platform.openai.com/storage/files/

# Step 2

## And now time to Fine-tune!

In [18]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4.1-nano-2025-04-14",
    seed=42,
    hyperparameters={"n_epochs": 1, "batch_size": 1},
    suffix="pricer"
)

FineTuningJob(id='ftjob-8F1UHI18rbgGkL2BCdCumxJr', created_at=1769673686, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XZ6ltGhtnNbWYghUsjzhsS2c', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-Js35KiuAFgARS2Es9o1DZG', validation_file='file-AfCR9zBTapjJ4vjssunxVM', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [19]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-8F1UHI18rbgGkL2BCdCumxJr', created_at=1769673686, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XZ6ltGhtnNbWYghUsjzhsS2c', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-Js35KiuAFgARS2Es9o1DZG', validation_file='file-AfCR9zBTapjJ4vjssunxVM', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None)], has_more=False, object='list')

In [20]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [21]:
job_id

'ftjob-8F1UHI18rbgGkL2BCdCumxJr'

In [22]:
openai.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-8F1UHI18rbgGkL2BCdCumxJr', created_at=1769673686, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XZ6ltGhtnNbWYghUsjzhsS2c', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-Js35KiuAFgARS2Es9o1DZG', validation_file='file-AfCR9zBTapjJ4vjssunxVM', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [23]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-gAgtG67SiunESwaydbP6iFX3', created_at=1769673686, level='info', message='Validating training file: file-Js35KiuAFgARS2Es9o1DZG and validation file: file-AfCR9zBTapjJ4vjssunxVM', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-l3roykDrQCdca7tbSvEOfKv9', created_at=1769673686, level='info', message='Created fine-tuning job: ftjob-8F1UHI18rbgGkL2BCdCumxJr', object='fine_tuning.job.event', data={}, type='message')]

https://platform.openai.com/finetune


# Step 3

Test our fine tuned model

In [35]:
job = openai.fine_tuning.jobs.retrieve(job_id)

print(job.status)
print(job.fine_tuned_model)

succeeded
ft:gpt-4.1-nano-2025-04-14:personal:pricer:D3HPYEc4


In [41]:
fine_tuned_model_name = job.fine_tuned_model

print(fine_tuned_model_name)

ft:gpt-4.1-nano-2025-04-14:personal:pricer:D3HPYEc4


In [36]:
# The prompt

def test_messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
    ]

In [37]:
# Try this out

test_messages_for(test[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Excess V2 Distortion/Modulation Pedal  \nCategory: Music Pedals  \nBrand: Old Blood Noise  \nDescription: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  \nDetails: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.'}]

In [42]:
# The inference function


def gpt_4__1_nano_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name,
        messages=test_messages_for(item),
        max_tokens=7
    )
    return response.choices[0].message.content

In [43]:
print(test[0].price)
print(gpt_4__1_nano_fine_tuned(test[0]))

219.0
$249.00


In [44]:
evaluate(gpt_4__1_nano_fine_tuned, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$5 $58 $5 $45 $117 $100 $1099 $60 $17 $519 $302 $3 $100 $81 $32 $16 $16 $18 $170 $230 $59 $1 $65 $119 $229 $254 $96 $20 $71 $44 $22 $37 $240 $31 $10 $283 $204 $36 $59 $23 $153 $64 $46 $70 $57 $4 $39 $17 $58 $40 $11 $106 $162 $10 $237 $75 $17 $0 $150 $16 $163 $48 $31 $60 $111 $18 $11 $228 $37 $6 $14 $2 $41 $7 $48 $23 $95 $7 $9 $6 $70 $8 $29 $79 $20 $25 $98 $16 $5 $20 $19 $124 $7 $20 $1 $116 $4 $187 $86 $7 $21 $48 $60 $83 $530 $68 $12 $378 $3 $31 $45 $460 $84 $4 $54 $90 $9 $7 $57 $703 $22 $131 $60 $174 $15 $11 $47 $64 $69 $79 $180 $40 $129 $0 $525 $15 $166 $10 $102 $11 $74 $71 $35 $88 $47 $36 $9 $175 $150 $7 $4 $64 $20 $16 $129 $70 $96 $38 $597 $45 $16 $15 $104 $3 $611 $7 $412 $36 $10 $3 $11 $6 $168 $13 $32 $380 $9 $37 $39 $32 $67 $70 $0 $110 $6 $8 $83 $17 $35 $14 $7 $73 $32 $21 $60 $135 $246 $110 $39 $17 

In [ ]:
# 96.58 - mini 200
# 79.29 - mini 2000
# 82.26 - nano 2000
# 67.75 - nano 20,000